Exploratory Data Analysis

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.cloud import bigquery


In [4]:
from data_audit import print_audit_report, audit_lfp_dataset
from get_data import dataset_stats

dataset_stats_df = dataset_stats(bigquery.Client(project='neural-ds-fe73'))
sample_data = pd.read_parquet('sample_data.parquet')
report = audit_lfp_dataset(sample_data)
print_audit_report(report, dataset_stats_df)

/opt/anaconda3/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


LFP SAMPLE DATASET AUDIT REPORT

SAMPLE DATASET STATS
   Total trials: 5895
   Total sessions: 2
   WT sessions: 1
   FMR1 sessions: 1
   Mean trials/session: 2947.5
   Std trials/session: 2386.5
   Min trials/session: 1260
   Max trials/session: 4635
   # frequencies: 15
   # amplitudes: 5
   # freq-amp combos: 75

FULL DATASET STATS (BQ)
   Total trials: 77418
   Total sessions: 32
   WT sessions: 25
   FMR1 sessions: 7
   Mean trials/session: 2419.3
   Std trials/session: 1039.2
   Min trials/session: 1215
   Max trials/session: 4635
   # frequencies: 22
   # amplitudes: 5
   # freq-amp combos: 96

COMPARISON (SAMPLE vs FULL)
   Total trials: 5895 vs 77418 (7.6% of full)
   Total sessions: 2 vs 32 (6.2% of full)
   WT sessions: 1 vs 25 (4.0% of full)
   FMR1 sessions: 1 vs 7 (14.3% of full)
   Trials/session mean: 2947.5 vs 2419.3
   Trials/session std:  2386.5 vs 1039.2
   Trials/session min:  1260.0 vs 1215.0
   Trials/session max:  4635.0 vs 4635.0
   Frequencies covered: 15 of 2